Import lib.

In [10]:
import os
import pandas as pd
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

from sklearn import preprocessing
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
# from sklearn.datasets import load_files
# sys.path.append('..') 
# import helpers.data_mining_helpers as dmh

# import nltk
# nltk.download('punkt')
# import seaborn as sns
# from sklearn.decomposition import PCA
# from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
# from sklearn.preprocessing import binarize
# from sklearn.metrics.pairwise import cosine_similarity



Data input.

In [11]:
train_file_path = 'training.csv'
test_file_path = 'public_processed.csv'
df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)


df.shape : (8688526, 26)  |||  test_df.shape : (600182, 25)

In [16]:
training_df = df
# print(df.shape)
# print(test_df.shape)

# Rename the columns to match the previous code
# df.columns = ['txkey', 'locdt', 'loctm', 'chid', 'cano', 'locdt']
# test_df.columns = ['sentence', 'score', 'provider']

# Combine the two dataframes
# X = test_df
# display(test_df.head())
# display(df.head())
# Randomly select 3000 rows and drop duplicates and missing values
# X = X.sample(n=3000, random_state=42, replace=False)
# X.dropna(inplace=True)
# X.drop_duplicates(inplace=True)
# display(X)




label_encoder = LabelEncoder()
training_df['chid'] = label_encoder.fit_transform(training_df['chid'])
training_df['cano'] = label_encoder.fit_transform(training_df['cano'])
training_df['mchno'] = label_encoder.fit_transform(training_df['mchno'])
training_df['acqic'] = label_encoder.fit_transform(training_df['acqic'])
training_df['etymd']= label_encoder.fit_transform(training_df['etymd'])
training_df['mcc']= label_encoder.fit_transform(training_df['mcc'])
training_df['stocn']= label_encoder.fit_transform(training_df['stocn'])
training_df['scity']= label_encoder.fit_transform(training_df['scity'])
training_df['stscd']= label_encoder.fit_transform(training_df['stscd'])
training_df['hcefg']= label_encoder.fit_transform(training_df['hcefg'])
training_df['csmcu']= label_encoder.fit_transform(training_df['csmcu'])
##test
test_df['chid'] = label_encoder.fit_transform(test_df['chid'])
test_df['cano'] = label_encoder.fit_transform(test_df['cano'])
test_df['mchno'] = label_encoder.fit_transform(test_df['mchno'])
test_df['acqic'] = label_encoder.fit_transform(test_df['acqic'])
test_df['etymd']= label_encoder.fit_transform(test_df['etymd'])
test_df['mcc']= label_encoder.fit_transform(test_df['mcc'])
test_df['stocn']= label_encoder.fit_transform(test_df['stocn'])
test_df['scity']= label_encoder.fit_transform(test_df['scity'])
test_df['stscd']= label_encoder.fit_transform(test_df['stscd'])
test_df['hcefg']= label_encoder.fit_transform(test_df['hcefg'])
test_df['csmcu']= label_encoder.fit_transform(test_df['csmcu'])


Data preprocess.

In [13]:
y = training_df['label']
X = training_df.drop(['label','txkey'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

training & testing.

In [31]:



# df_sample = df.sample(n=300000, random_state=42)
# train_df, val_df = train_test_split(df_sample, test_size=0.2, random_state=42)

# # Define the features and target variable
# #featues 前面preprocess先做好這邊在註解掉
# features = [ 'locdt','loctm','contp','etymd','mcc','conam','ecfg','insfg','iterm','bnsfg','flam1','stocn','scity','ovrlt','flbmk','hcefg','csmcu','csmam','flg_3dsmk'] 
# target = 'label'

# # Split the training and validation datasets into features and target variable
# X_train = train_df[features]
# y_train = train_df[target]
# X_val = val_df[features]
# y_val = val_df[target]

# # Train the model using LightGBM

# # params = {
# #     'objective': 'binary',
# #     'metric': 'binary_logloss',
# #     'boosting_type': 'dart',
# #     'num_leaves': 50,  
# #     'learning_rate': 0.01,  
# #     'feature_fraction': 0.8  
# # }
# params = {'objective': 'binary',
#           'boosting_type': 'gbdt',
#           'metric': 'auc',
#           'learning_rate': 0.007,
#           'num_leaves': 2**8,
#           'max_depth': -1,
#           'tree_learner':'serial',
#           'colsample_bytree': 0.5,
#           'subsample_freq':1,
#           'subsample':0.7,
#           'n_estimators':10000,
#           'min_data_in_leaf': 106,
#           'max_bin':255,
#           'verbosity': -1,
#           'early_stopping_rounds':100,
#          }


# train_data = lgb.Dataset(X_train, label=y_train)
# val_data = lgb.Dataset(X_val, label=y_val)

# model = lgb.train(params, train_data, num_boost_round=10000, valid_sets=[train_data, val_data])
# model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data, val_data], early_stopping_rounds=50)

categories = ['chid', 'cano', 'contp', 'etymd', 'mchno', 'acqic',
              'mcc',  'ecfg', 'insfg',  'bnsfg', 'stocn', 'scity',
              'stscd', 'ovrlt', 'flbmk', 'hcefg', 'csmcu', 'flg_3dsmk']

params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.005,
    'n_estimators': 3000,
    'boosting_type': 'gbdt',
    'categorical_feature': categories,
}   

train_data = lgb.Dataset(X_train, label=y_train,
                         categorical_feature=categories)
test_data = lgb.Dataset(X_test, label=y_test,
                        reference=train_data)

bst = lgb.train(params, train_data,  valid_sets=[test_data])

c:\Users\lo\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\lo\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1873: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
c:\Users\lo\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightgbm\basic.py:1893: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set=chid,cano,contp,etymd,mchno,acqic,mcc,ecfg,insfg,bnsfg,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,flg_3dsmk, categorical_column=2,3,4,5,6,7,8,10,11,13,15,16,17,18,19,20,21,23 will be ignored. Current value: categorical_feature=chid,cano,contp,etymd,mchno,acqic,mcc,ecfg,insfg,bnsfg,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,flg_3dsmk
[LightGBM] [Info] Number of positive: 25694, number of negative: 6925126
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 39625
[LightGBM] [Info] Number of data points in the train set: 6950820, number of used features: 24


LightGBMError: bin size 15964 cannot run on GPU

validation.

In [46]:
# y_pred_prob = model.predict(X_val)
# y_pred = [1 if x >= 0.5 else 0 for x in y_pred_prob]  # Assuming binary classification, adjust as needed
# # y_pred = model.predict(X_val)
# # print(y_pred)
# # # Get the true labels from the validation set
# # y_true = val_data.label

# # # Calculate F1 score
# f1 = f1_score(y_val, y_pred)
# print(f'F1 Score on Validation Set: {f1}')

# y_pred = pd.Series(bst.predict(X_test, num_iteration=bst.best_iteration)>0.5).astype(int)

# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# print("Classification Report:")
# print(classification_report(y_test, y_pred))

F1 Score on Validation Set: 0.0


predict.

In [22]:
# display(training_df.head())
# display(X_test.head())
# display(test_df.head())
predict_data = test_df.drop(['txkey'], axis=1)
y_pred = pd.Series(bst.predict(predict_data, num_iteration=bst.best_iteration)>0.5).astype(int)
# pridict = pd.DataFrame(test_df['txkey'] )
# pridict['pred'] = y_pred # put here
# display(pridict)

In [28]:
# test_df = pd.read_csv(test_file_path)
pridict = pd.DataFrame(test_df['txkey'] )
pridict['pred'] = y_pred 
# display(y_pred )
# print(np.nonzero(y_pred))
# print(y_pred[1771])
# display(pridict.iloc[1900])
display(pridict)

,txkey,pred
0,a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6c...,0
1,16c4880500059e01553789be11bbb50753b7acaae7b95b...,0
2,623c56be3bee87724e3d119c271d9ed098eeda84233183...,0
3,250da12187059cf6e3a3066656a2919d08ceb8207efd55...,0
4,4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843...,0
...,...,...
600177,741a391b2c2114491d1d18acb1a892341b9280e1e3529f...,0
600178,76f613595631fb35938bf10f5b7abe3f60a3d34ccf5f44...,0
600179,c32e846a4469e4869e2df8f8e160fba4e829b6beebb44d...,0
600180,d641cd25a830de759cd55b4a643d71eadb770bc917aed3...,0


output.

In [29]:
pridict.to_csv('output.csv', index=False)